In [1]:
from crypto import cryptopuf
import numpy as np
from pypuf.pypuf import io
from pypuf.pypuf.metrics import uniqueness, bias
from pypuf.pypuf import attack
from pypuf.pypuf import metrics

2024-11-10 18:06:18.306500: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 18:06:18.314818: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 18:06:18.323910: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 18:06:18.326443: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 18:06:18.333445: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Uniqueness evaluation
seed = 31415
np.random.seed(seed)
puf_instances = []
for i in range(5):  # Create 5 PUF instances
    new_key = np.random.bytes(16).hex()  # New random key for each instance
    nonce = "000000000000000000000000"  # 96-bit nonce for TinyJAMBU
    crypto_puf = cryptopuf.CryptoPUF_TinyJAMBU(challenge_length=128, key=new_key, nonce=nonce)
    print(f"key is {new_key}")
    puf_instances.append(crypto_puf)

# Evaluate uniqueness using the `uniqueness()` function
uniqueness_score = uniqueness(puf_instances, seed=seed, N=1000)
print(f"Uniqueness score of the responses: {uniqueness_score}")

# Bias evaluation
bias_score = bias(puf_instances[0], seed=seed)
print(f"bias is {bias_score}")


key is 3e5e54a5a71836fce336d8ac0d68ec14
key is 650e80f8a8356891f28982e2866d7080
key is 068ab8e8e35414031efdd6d2553f17bd
key is 0e02997c66cc2af2706ff99e7056385d
key is 883bbda4d30cdbbda69e7e663a5c1680
Uniqueness score of the responses: [0.9796]
bias is -0.004


In [3]:
N = 50000
print(f"Training with {N} CRPs...")

# Generate CRPs for the current iteration
crps = io.ChallengeResponseSet.from_simulation(crypto_puf, N=N, seed=2)

# Create and train the MLPAttack202 attack with the generated CRPs
attack_instance = attack.LRAttack2021(crps, seed=3, k=4, bs=1000, lr=.001, epochs=100)
attack_instance.fit()

# Evaluate the model's similarity score
model = attack_instance.model
score = metrics.similarity(crypto_puf, model, seed=4)
print(f"Predication accuracy of the trained LR model is with 50000 CRPs: {score}")


Training with 50000 CRPs...
Epoch 1/100


I0000 00:00:1731279979.852934  210830 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:03:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-11-10 18:06:19.892604: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.5057 - loss: 0.6934 - val_accuracy: 0.5160 - val_loss: 0.6936
Epoch 2/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5157 - loss: 0.6919 - val_accuracy: 0.4940 - val_loss: 0.6931
Epoch 3/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 972us/step - accuracy: 0.5203 - loss: 0.6911 - val_accuracy: 0.4960 - val_loss: 0.6928
Epoch 4/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 989us/step - accuracy: 0.5227 - loss: 0.6904 - val_accuracy: 0.4900 - val_loss: 0.6926
Epoch 5/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 925us/step - accuracy: 0.5257 - loss: 0.6896 - val_accuracy: 0.4860 - val_loss: 0.6927
Epoch 6/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 914us/step - accuracy: 0.5275 - loss: 0.6888 - val_accuracy: 0.5020 - val_loss: 0.6930
Epoch 7/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 929us/step - accuracy: 0.5287 - loss: 0.6880 - val_accuracy: 0.4920 - val_loss: 0.6934
Epoch 8/100
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 906us/step - accuracy: 0.5327 - loss: 0.6874 - val_accuracy: 0.4720 - v

In [4]:
# Create and train the MLPAttack202 attack with the generated CRPs
attack_instance = attack.MLPAttack2021(
crps, seed=3, net=[2 ** 4, 2 ** 5, 2 ** 4],
epochs=30, lr=.001, bs=1000, early_stop=.08
)
attack_instance.fit()

# Evaluate the model's similarity score
model = attack_instance.model
score = metrics.similarity(crypto_puf, model, seed=4)
print(f"Predication accuracy of the trained LR model is with 50000 CRPs: {score}")

Epoch 1/30


/home/yimin/.local/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


50/50 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.5013 - loss: 0.7286 - val_accuracy: 0.4740 - val_loss: 0.6985
Epoch 2/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5119 - loss: 0.6931 - val_accuracy: 0.5060 - val_loss: 0.6965
Epoch 3/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5191 - loss: 0.6918 - val_accuracy: 0.5100 - val_loss: 0.6965
Epoch 4/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5271 - loss: 0.6907 - val_accuracy: 0.5080 - val_loss: 0.6971
Epoch 5/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5308 - loss: 0.6896 - val_accuracy: 0.5160 - val_loss: 0.6981
Epoch 6/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5374 - loss: 0.6886 - val_accuracy: 0.5160 - val_loss: 0.6992
Epoch 7/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5411 - loss: 0.6876 - val_accuracy: 0.5100 - val_loss: 0.7001
Epoch 8/30
50/50 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5442 - loss: 0.6866 - val_accuracy: 0.4960 - val_loss: 0.7009
Epo